<a href="https://colab.research.google.com/github/ighoshsubho/Diversion_Blinders/blob/main/Finetune_Text_Generation_Model_For_YBS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Import all the libraries


---



Here we import all the required libraries and then check the gpu being used in the notebook you are using! First use kaggle.json to download the dataset. 

[Link on how to download kaggle datasets on colab.](https://www.kaggle.com/general/74235)

In [ ]:
! pip install -q kaggle

In [ ]:
from google.colab import files

In [ ]:
! mkdir ~/.kaggle

In [ ]:
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"ighoshsubho","key":"d5b79893f85c99c2d6114b92d19ed131"}'}

In [ ]:
! cp kaggle.json ~/.kaggle/

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets list

In [ ]:
!pip uninstall -qqy torch torchvision torchtext torchaudio fastai
!pip install -qq torch==1.9.0 pytorch-lightning==1.7.7 aitextgen==0.6.0 

In [ ]:
!pip3 install -q aitextgen==0.5.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 572.3/572.3 KB 35.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

!nvidia-smi

Fri Feb 24 05:54:19 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P0    27W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

##Downloading the dataset:

We are now using bash command to download the dataset.

Data Credit - https://github.com/krea-ai/open-prompts

In [ ]:
!wget -O prompts.csv https://raw.githubusercontent.com/krea-ai/open-prompts/main/data/1k.csv

--2023-02-24 06:31:19--  https://raw.githubusercontent.com/krea-ai/open-prompts/main/data/1k.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1027887 (1004K) [text/plain]
Saving to: ‘prompts.csv’

prompts.csv         100%[===================>]   1004K  --.-KB/s    in 0.007s  

2023-02-24 06:31:19 (144 MB/s) - ‘prompts.csv’ saved [1027887/1027887]



In [ ]:
!ls

input_text_cleaned.txt	kaggle.json  prompts.csv  sample_data


In [ ]:
prompts = pd.read_csv("prompts.csv")

NameError: ignored

In [ ]:
prompts

NameError: ignored

##Working on prompts

We want prompts where no. of words are greater than 6 so we perform the following operation.

In [ ]:
prompts_gt6 = prompts.loc[prompts.prompt.str.split(' ').str.len() > 6]

NameError: ignored

Now we install a package called aitextgen which will immensely help us in finetuning our GPT Neo model.

In [ ]:
! pip install aitextgen -q

In [ ]:
#You can give model according to your needs for finetuning

model="EleutherAI/gpt-neo-125M"

We are just taking the prompt coloumn!

In [ ]:
prompts_gt6 = prompts_gt6.drop('raw_data', axis = 1).reset_index(drop=True)

In [ ]:
prompts_gt6.to_csv("input_text_cleaned.txt", columns=["prompt"], header=False, index=False)

#Loading the model for finetuning!

Now we can load the model by importing TokenDataset first to load the data and then importing the aitextgen to load the model.

In [ ]:
from aitextgen.TokenDataset import TokenDataset

In [ ]:
data = TokenDataset('./input_text_cleaned.txt', line_by_line=True)

  0%|          | 0/896 [00:00<?, ?it/s]

INFO:aitextgen.TokenDataset:Encoding 896 sets of tokens from ./input_text_cleaned.txt.


In [ ]:
!pip install cloud-tpu-client==0.10 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl
!pip install torch==1.9.0+cu111 torchvision==0.10.0+cu111 torchtext==0.10.0 -f https://download.pytorch.org/whl/cu111/torch_stable.html

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: torch_xla-1.9-cp37-cp37m-linux_x86_64.whl is not a supported wheel on this platform.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/whl/cu111/torch_stable.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 GB 838.1 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.2/23.2 MB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 64.9 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 1.9.0
    Uninstalling torch-1.9.0:
      Successfully uninstalled torch-1.9.0


In [ ]:
from aitextgen import aitextgen

In [ ]:
# ai = aitextgen(tf_gpt2="124M",  to_gpu=True)
ai = aitextgen(model = model,  to_gpu=True)

NameError: ignored

##Train the model with your dataset!

Now we are training the model with custom dataset!

In [ ]:
ai.train('input_text_cleaned.txt',
         line_by_line=True,
         from_cache=False,
         num_steps=500,
         generate_every=100,
         save_every=500,
         save_gdrive=False,
         learning_rate=1e-3,
         fp16=False,
         batch_size=1, 
         )

##Save the model

Either it can be in hugging face or it can be local to your machine.

In [ ]:
ai.save()

In [ ]:
prompt_ai = aitextgen(model_folder = '.', to_gpu=True)

In [ ]:
print(prompt_ai.generate(prompt = "astronaut "))

##Congratulations

You have finetuned a model for Stable Diffusion prompt!

In [ ]:
ai.save_for_upload('sd-prompt-generator-gpt-neo')

##Uploading the model to Hugging Face Model Hub

Upload the model and use it in YBS to see magic!

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
!sudo apt-get install git-lfs

In [ ]:
! pip install --upgrade huggingface_hub -q

In [ ]:
from huggingface_hub import HfApi

In [ ]:
api = HfApi()

In [ ]:
api.upload_folder(
    folder_path="./sd-prompt-generator-gpt-neo",
    path_in_repo = ".",
    repo_id="Amrrs/sd-prompt-generator-gpt-neo",
    repo_type="model"
)

##Boom!

You have successfully uploaded your model on hugging face model hub and now you can use it for community contributions!

Github Repo - https://github.com/Aditya007Datta/yourbrandstories